In [ ]:
import pandas as pd
import numpy as np
import os

# Define input and output directory mapping
input_output_mapping = {
    "train": "train_small_timestep",
    "testing": "testing_small_timestep",
    "validation": "validation_small_timestep"
}

# Process each input-output directory pair
for input_dir, output_dir in input_output_mapping.items():
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Walk through all files in input directory
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.endswith(".csv"):
                # Build full input path
                input_path = os.path.join(root, filename)

                # Calculate relative path for maintaining directory structure
                if root == input_dir:
                    # For files in top-level directory
                    output_subdir = output_dir
                else:
                    # For files in subdirectories
                    rel_path = os.path.relpath(root, input_dir)
                    output_subdir = os.path.join(output_dir, rel_path)

                # Ensure output subdirectory exists
                os.makedirs(output_subdir, exist_ok=True)

                # Build full output path
                output_path = os.path.join(output_subdir, filename)

                # Read and process the data
                try:
                    df = pd.read_csv(input_path)

                    # Linear interpolation to expand from 100 to 200 points
                    new_time = np.linspace(df['Time (s)'].min(), df['Time (s)'].max(), 200)
                    new_df = pd.DataFrame({'Time (s)': new_time})

                    # Interpolate each column
                    for column in df.columns[1:]:
                        new_df[column] = np.interp(new_time, df['Time (s)'], df[column])

                    # Save interpolated data
                    new_df.to_csv(output_path, index=False)
                    print(f"Processed: {input_path} → {output_path}")

                except Exception as e:
                    print(f"Failed to process: {input_path} - Error: {str(e)}")

print("All directories processed successfully!")